# multiprocessing Intro

### Python multiprocessing vs multiprocess
**🔹 multiprocessing (✅ Built-in)**
- Standard library module in Python (since Python 2.6)
- Enables true parallelism using multiple processes
- Bypasses Python’s GIL, making it ideal for CPU-bound tasks
- Comes with tools like:
    - `Process`, `Queue`, `Pool`, `Pipe`, `Manager`

**🔸 multiprocess (🔁 Third-party package)**
- An external library (install via pip install multiprocess)
- A fork of multiprocessing, developed by the pathos team
- Supports more advanced serialization using dill instead of pickle
- Meaning: it can handle closures, lambdas, and more complex objects
- API is almost identical to multiprocessing

**Basic Example**

In [1]:
from multiprocess import Process, Manager
import time

def square(x, res):
    res[x] = x ** 2
    print(f"[Child] square[{x}] = {x**2}, res = {dict(res)}")

if __name__ == "__main__":
    with Manager() as manager:
        result = manager.dict()

        p1 = Process(target=square, args=(1, result))
        p2 = Process(target=square, args=(22, result))

        p1.start()
        p2.start()

        p1.join()
        p2.join()

        print(f"[Main] result = {dict(result)}")
    print("All processing done")


[Main] result = {1: 1, 22: 484}
All processing done


**✅ Important Points**
- Each process in multiprocessing runs in its own memory space, so regular Python objects like dict aren't shared across processes.
    - ✅ To share data, use Manager().dict() — a shared dictionary proxy accessible by all child processes.
-⚠️ Avoid running multiprocessing code in interactive environments (like IDLE, Jupyter, or REPLs), as it may behave unexpectedly.
    - ✅ Prefer running your multiprocessing code as a standalone script (.py file) via terminal or command prompt.
- 🔁 Common Pitfall:
    - Always wrap multiprocessing code inside the if __name__ == "__main__": guard.
    - ❌ Missing this on Windows/macOS can lead to unexpected behavior or infinite process spawning.
- 🧪 In notebooks, the built-in multiprocessing module may not function as expected.
    - ✅ To work around this, use the multiprocess package instead.
    - ✅ If running the same code as a script, you can safely switch back to:
    - `from multiprocessing import Process, Manager`


## What is Manager()?
- Manager() is a special helper in the multiprocessing module that allows you to create shared objects like: dict, list, Namespace, Lock, Queue, etc.
- These shared objects can be safely accessed and modified across multiple processes.

**🔧 Why do you need it?**
- In multithreading, threads share memory. But in multiprocessing, each process has its own memory space. So, if you do this: `result = {}`
and then pass it to two Processes — it won’t work. Each process gets a copy, not the original.

- To fix that, you use:
    -   ```python
        from multiprocessing import Manager
        result = Manager().dict()
        ```
- Now result is a proxy object, backed by a server process that all child processes can talk to safely.

**🔁 Behind the Scenes**
- Manager() spawns a server process internally.
- When you use manager.dict() or manager.list(), it returns proxy objects.
- These proxies handle inter-process communication (IPC) and synchronize access.